In [1]:
#
from itertools import groupby
import sys

# from devolucao import get_df_devol
sys.path.append("..")
import DB
import workdays
import datetime
import pandas as pd
import numpy as np
import carteira_ibov
import taxas
import os
pd.options.mode.chained_assignment = None  # default='warn'
import config
import psycopg2
import pandas as pd
import workdays
import pyodbc
import ibotz
#
holidays_br = workdays.load_holidays("BR")
holidays_b3 = workdays.load_holidays("B3")

dt = datetime.date.today()
dt = workdays.workday(dt, -1, holidays_br)
vcto_0 = dt
# dt_pos = workdays.workday(dt, -1, holidays_br)


def rate_format(a):
    if float("{:.2f}".format(a).split('.')[-1])==0.0:
        return str(int(a))
    else:
        return    str(round(float(a),2))



def venc_format(a):
    if type(a)==str:
        return datetime.datetime.strptime(a,'%Y-%m-%d').strftime('%Y%m%d')
    else:
        return  a.strftime('%Y%m%d')


import os

# folder path
dir_path = f"G:\Trading\K11\Aluguel\Controle\{dt.strftime('%d-%m-%Y')}"

# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)
boleta_final = pd.DataFrame()
for x in res:
    path = dir_path + "\\" + x
    print(x)
    boleta = pd.read_excel(dir_path + "\\" + x)
    boleta = boleta.rename(columns={
    'str_fundo':'ALOCACAO',
    'str_papel':'codigo',
    })
    boleta['str_tipo_registro'] = boleta['str_tipo_registro'].fillna("R")
    parse_reg ={
        'R':'BALCAO',
        'N':'E1'
    }
    parse_brokers={
        'Merrill Lynch':'Bank of America'
    }
        
    total = boleta.groupby(['ALOCACAO','codigo']).agg({'dbl_quantidade':sum}).reset_index()

    espelho_doador = pd.read_excel(f"G:\Trading\K11\Aluguel\Arquivos\Doar\Quebra-Dia\K11_Quebra_complete_{dt.strftime('%d-%m-%Y')}.xlsx",names=['ALOCACAO','mesa','str_estrategia','codigo','to_lend','total','prop'])

    # boleta['dbl_taxa'] = boleta['dbl_taxa'].apply(lambda x: str(round(float(x),0)) if x.is_integer() else  )
    doador = boleta[boleta['str_tipo']=='D']
    if not doador.empty:

        # doador['dte_datavencimento'] =doador['dte_datavencimento'].apply(lambda x: datetime.datetime.strptime(x,'%d/%m/%y')if type(x)==str else x)
        doador  = doador.merge(espelho_doador,how='left',on=['ALOCACAO','codigo']).fillna(1)

        
        doador['dbl_quantidade'] = doador['dbl_quantidade'].apply(lambda x: -abs(x))
        
        doador['final doador']  = round(doador['dbl_quantidade']*doador['prop'],0)
        
        check = doador.groupby(['ALOCACAO','codigo']).agg({'final doador':'sum'}).reset_index()
        check  = check.merge(total,how='inner',on=['ALOCACAO','codigo'])
        check['ajuste'] = check['final doador'] - check['dbl_quantidade']
        check = check[check['ajuste']!=0]
        doador['str_tipo_registro'] = doador['str_tipo_registro'].map(parse_reg)
        
        doador['SERIE'] = doador.apply(lambda row: row['codigo'] + "-" +row['str_tipo']+"-"+ rate_format(float(row['dbl_taxa'])).replace('.',',') + "-"+ venc_format(row['dte_datavencimento'])+"-"+row['str_tipo_registro'],axis=1 )

        doador = doador[['ALOCACAO','mesa','str_estrategia','codigo','SERIE','final doador']]
        doador['CLEARING'] = 'Bradesco'
        doador['CONTRA'] = x.split('_')[0].replace('Necton','Concordia').replace('UBS','Link').replace('Merrill Lynch','Bank of America')
        doador['TIPO'] = 'Emprestimo RV'
        doador['CODIGO'] = doador['SERIE'].apply(lambda x: x[:4])
        doador['MESA'] = doador['mesa']
        doador['ESTRATEGIA'] = doador['str_estrategia']
        doador['NOTIONAL'] = doador['final doador']
        doador['PREMIO'] = 0
        doador = doador[
        [
            "ALOCACAO",
            "MESA",
            "ESTRATEGIA",
            "CLEARING",
            "CONTRA",
            "TIPO",
            "CODIGO",
            "SERIE",
            "NOTIONAL",
            "PREMIO"
        ]
        ]
        
        boleta_final  = pd.concat([doador[doador['NOTIONAL']!=0],boleta_final])

  

import os

# folder path
dir_path = f"G:\Trading\K11\Aluguel\Controle\{dt.strftime('%d-%m-%Y')}"

# list to store files
res = []

resto_final = pd.DataFrame()
# Iterate directory
# boleta_final = pd.DataFrame()
for path in os.listdir(dir_path):
	# check if current path is a file
	if os.path.isfile(os.path.join(dir_path, path)):
		res.append(path)



for x in res:
	path = dir_path + "\\" + x
	print(x)
	boleta = pd.read_excel(dir_path + "\\" + x)
	boleta = boleta.rename(columns={
	'str_fundo':'ALOCACAO',
	'str_papel':'codigo',
	})
	boleta['str_tipo_registro'] = boleta['str_tipo_registro'].fillna("R")
	parse_reg ={
		'R':'BALCAO',
		'N':'E1'
	}
		
	total = boleta.groupby(['ALOCACAO','codigo']).agg({'dbl_quantidade':sum}).reset_index()

	espelho_doador = pd.read_excel(f"G:\Trading\K11\Aluguel\Arquivos\Doar\Quebra-Dia\K11_Quebra_complete_{dt.strftime('%d-%m-%Y')}.xlsx",names=['ALOCACAO','mesa','str_estrategia','codigo','to_lend','total','prop'])

	# boleta['dbl_taxa'] = boleta['dbl_taxa'].apply(lambda x: str(round(float(x),0)) if x.is_integer() else  )

	tomador = boleta[boleta['str_tipo']=='T']
	
	
	if not tomador.empty:
	# tomador = boleta[boleta['str_tipo']=='T']
		
		mapa = pd.read_excel(f"G:\Trading\K11\Aluguel\Arquivos\Main\main_v2_{dt.strftime('%Y-%m-%d')}.xlsx")

		espelho_t = mapa.groupby(['fundo','codigo']).agg({'position':sum,'to_borrow_0':sum,'to_borrow_1':sum}).reset_index()
		espelho_t = espelho_t.loc[(espelho_t['position']<0)|(espelho_t['to_borrow_0']<0)|(espelho_t['to_borrow_1']<0)].rename(columns={'position':'total','to_borrow_0':'janela','to_borrow_1':'dia'})

		# quebra_t = mapa[mapa['to_borrow_1']<0][['fundo','mesa','str_estrategia','codigo','to_borrow_1']]
		quebra_t = mapa[['fundo','mesa','str_estrategia','codigo','position','to_borrow_0','to_borrow_1']]
		quebra_t = quebra_t.merge(espelho_t,on=['fundo','codigo'],how='inner')


		## tomar pro dia
		dia = quebra_t[quebra_t['to_borrow_1']!=0]
		dia['prop_borrow_1']= dia.apply(lambda row: min(row['to_borrow_1']/row['dia'],1),axis=1)
		## janela

		janela = quebra_t[quebra_t['to_borrow_0']!=0]
		janela['prop_borrow_0']= janela.apply(lambda row: min(row['to_borrow_0']/row['janela'],1),axis=1)

		## position
		pos = quebra_t[(quebra_t['position']!=0) & (quebra_t['total']!=0)]
		pos['prop_total']= pos.apply(lambda row: min(row['position']/row['total'],1),axis=1)

		aux_prop = dia.groupby(['fundo','codigo']).sum().reset_index()

		aux_prop['ajuste'] = 1 - aux_prop['prop_borrow_1'] 
		aux_total = aux_prop
		aux_prop = aux_prop[aux_prop.ajuste!=0]
		dia  = dia.sort_values('prop_borrow_1')

		for i, row in dia.iterrows():
			for a, row_x in aux_prop.iterrows():
				if (row['codigo'] == row_x['codigo']) & (row['fundo'] == row_x['fundo'])&(row_x['ajuste']!=0):
					dia.loc[i,'prop_borrow_1'] = row['prop_borrow_1']+ aux_prop.loc[a,'ajuste'] 
					dia.loc[a,'ajuste'] = 0
		dia = dia[['fundo','mesa','str_estrategia','codigo','prop_borrow_1']]
		dia.columns = ['ALOCACAO','mesa','str_estrategia','codigo','prop']


		aux_prop = janela.groupby(['fundo','codigo']).sum().reset_index()

		aux_prop['ajuste'] = 1 - aux_prop['prop_borrow_0'] 
		aux_total = aux_prop
		aux_prop = aux_prop[aux_prop.ajuste!=0]
		janela  = janela.sort_values('prop_borrow_0')

		for i, row in janela.iterrows():
			for a, row_x in aux_prop.iterrows():
				if (row['codigo'] == row_x['codigo']) & (row['fundo'] == row_x['fundo'])&(row_x['ajuste']!=0):
					janela.loc[i,'prop_borrow_0'] = row['prop_borrow_0']+ aux_prop.loc[a,'ajuste'] 
					janela.loc[a,'ajuste'] = 0
		janela = janela[['fundo','mesa','str_estrategia','codigo','prop_borrow_0']]
		janela.columns = ['ALOCACAO','mesa','str_estrategia','codigo','prop']



		aux_prop = pos.groupby(['fundo','codigo']).sum().reset_index()

		aux_prop['ajuste'] = 1 - aux_prop['prop_total'] 
		aux_total = aux_prop
		aux_prop = aux_prop[aux_prop.ajuste!=0]
		pos  = pos.sort_values('prop_total')

		for i, row in pos.iterrows():
			for a, row_x in aux_prop.iterrows():
				if (row['codigo'] == row_x['codigo']) & (row['fundo'] == row_x['fundo'])&(row_x['ajuste']!=0):
					pos.loc[i,'prop_total'] = row['prop_total']+ aux_prop.loc[a,'ajuste'] 
					aux_prop.loc[a,'ajuste'] = 0
		pos = pos[['fundo','mesa','str_estrategia','codigo','prop_total']]
		pos.columns = ['ALOCACAO','mesa','str_estrategia','codigo','prop']

		col_tomador = tomador.columns
		tomador = tomador.merge(janela,on=['ALOCACAO','codigo'],how='left').fillna(0)
		resto = tomador[tomador['mesa']==0]
		tomador = tomador[tomador['mesa']!=0]

		tomador_1 = resto[col_tomador].merge(dia,on=['ALOCACAO','codigo'],how='left').fillna(0)

		resto = tomador_1[tomador_1['mesa']==0]

		tomador = pd.concat([tomador_1[tomador_1['mesa']!=0],tomador])

		tomador_1 = resto[col_tomador].merge(dia,on=['ALOCACAO','codigo'],how='left').fillna(0)

		resto = tomador_1[tomador_1['mesa']==0]

		tomador = pd.concat([tomador_1[tomador_1['mesa']!=0],tomador])
		
		tomador_1 = resto[col_tomador].merge(pos,on=['ALOCACAO','codigo'],how='left').fillna(0)

		resto = tomador_1[tomador_1['mesa']==0]

		tomador = pd.concat([tomador_1[tomador_1['mesa']!=0],tomador])

		tomador['dbl_quantidade'] = tomador['dbl_quantidade'].apply(lambda x: abs(x))

		tomador['final tomador']  = round(tomador['dbl_quantidade']*tomador['prop'],0)

		check_t = tomador.groupby(['ALOCACAO','codigo']).agg({'final tomador':'sum'}).reset_index()
		check_t  = check_t.merge(total,how='inner',on=['ALOCACAO','codigo'])
		check_t['ajuste'] = check_t['final tomador'] - check_t['dbl_quantidade']
		check_t = check_t[check_t['ajuste']!=0]
		tomador['str_tipo_registro'] = tomador['str_tipo_registro'].map(parse_reg)
		tomador['SERIE'] = tomador.apply(lambda row: row['codigo'] + "-" +row['str_tipo']+"-"+ rate_format(float(row['dbl_taxa'])).replace('.',',') + "-"+ venc_format(row['dte_datavencimento'])+"-"+row['str_tipo_registro'],axis=1 )

		tomador = tomador[['ALOCACAO','mesa','str_estrategia','codigo','SERIE','final tomador']]
		tomador['CLEARING'] = 'Bradesco'
		tomador['CONTRA'] = x.split('_')[0].replace('Necton','Concordia').replace('UBS','Link').replace('Bofa','Bank of America')
		tomador['TIPO'] = 'Emprestimo RV'
		tomador['CODIGO'] = tomador['SERIE'].apply(lambda x: x[:4])
		tomador['MESA'] = tomador['mesa']
		tomador['ESTRATEGIA'] = tomador['str_estrategia']
		tomador['NOTIONAL'] = tomador['final tomador']
		tomador['PREMIO'] = 0
		tomador = tomador[
		[
			"ALOCACAO",
			"MESA",
			"ESTRATEGIA",
			"CLEARING",
			"CONTRA",
			"TIPO",
			"CODIGO",
			"SERIE",
			"NOTIONAL",
			"PREMIO"
		]
		]
		boleta_final  = pd.concat([tomador[tomador['NOTIONAL']!=0],boleta_final])
		resto_final = pd.concat([resto,resto_final])







Bradesco_dia_20240109.xlsx
Bradesco_janela_20240109.xlsx
BTG_trade_20240109.xlsx
XP_trade_20240109.xlsx
Bradesco_dia_20240109.xlsx
Bradesco_janela_20240109.xlsx
BTG_trade_20240109.xlsx
XP_trade_20240109.xlsx


In [2]:
resto_final


,Unnamed: 0,dte_databoleta,dte_data,str_corretora,str_tipo,dte_datavencimento,dbl_taxa,str_reversivel,str_tipo_registro,str_modalidade,str_tipo_comissao,dbl_valor_fixo_comissao,dbl_quantidade,str_status,ALOCACAO,mesa,str_estrategia,codigo,prop


In [3]:
boleta_final = boleta_final.drop_duplicates()

In [4]:

boleta_final['SIDE'] = boleta_final['NOTIONAL'].apply(lambda x: "B" if x>0 else "S")


In [5]:
boleta_final['NOTIONAL'] = boleta_final['NOTIONAL'].apply(lambda x: str(x))
boleta_final['PREMIO'] = boleta_final['PREMIO'].apply(lambda x: str(x))

In [6]:
boleta_final.loc[(boleta_final['MESA']==1)]

,ALOCACAO,MESA,ESTRATEGIA,CLEARING,CONTRA,TIPO,CODIGO,SERIE,NOTIONAL,PREMIO,SIDE


In [6]:
boleta_final.loc[(boleta_final['MESA']!=1)].to_clipboard()

In [17]:
boleta_final.to_clipboard()

In [9]:
import ibotz
ibotz.df_to_ibotz("joao.ramalho","Kapitalo@03",boleta_final.loc[(boleta_final['MESA']!=1)].drop_duplicates())

<Response [200]>

In [33]:
import pandas as pd

# Define the source file name
source_file = r"C:\Users\joao.ramalho\Downloads\KAPITALO"+' 07NOV2023 BOVA11 x WINZ23.xlsx"

# Read data from the source Excel file
df_source = pd.read_excel(source_file)

# Apply transformations to the data
# Extract "CODIGO" and "SERIE" from the 3rd column
df_source["CODIGO"] = df_source.iloc[:, 2].str.split(" ").str[0]
df_source["SERIE"] = df_source.iloc[:, 2].str.split(" ").str[1]

# Extract "LOTE" from the 11th column and "PRECO" from the 12th column
df_source["LOTE"] = df_source.iloc[:, 10]
df_source["PRECO"] = df_source.iloc[:, 11]

# Select only the relevant columns
df_transformed = df_source[["CODIGO", "SERIE", "LOTE", "PRECO"]]


df_transformed = df_transformed.dropna()

df_transformed['TIPO'] = df_transformed['CODIGO'].apply(lambda x: 'Futuro' if 'WIN' in x else 'Acao')
df_transformed['SERIE'] = df_transformed.apply(lambda x: x['CODIGO'][-3:] if 'WIN' in x['CODIGO'] else x['CODIGO']+' BZ EQUITY',axis=1)
df_transformed['CODIGO'] = df_transformed.apply(lambda x: 'WIN' if 'WIN' in x['CODIGO'] else x['CODIGO'][0:4],axis=1)


df_transformed['FUNDO'] = 'KAPITALO KAPPA MASTER FIM'
df_transformed['MESA'] = 'Kapitalo 11.1'
df_transformed['ESTRATEGIA'] = 'CashCarry5'
df_transformed['CLEARING'] = 'Bradesco'
df_transformed['CORRETORA'] = 'Liquidez'
recap = df_transformed[['FUNDO', 'MESA','ESTRATEGIA', 'CLEARING', 'CORRETORA','CODIGO', 'SERIE', 'LOTE', 'PRECO']]

recap_check = recap.copy()
recap_check['PROD'] = recap['LOTE']*recap['PRECO']

recap_check = recap_check.groupby(['CODIGO']).agg({'LOTE':sum,'PROD':sum}).reset_index()

recap_check['MEAN'] = recap_check['PROD']/recap_check['LOTE']

bova_mean = recap_check.loc[recap_check['CODIGO']=='BOVA','MEAN'].item()
win_mean = recap_check.loc[recap_check['CODIGO']=='WIN','MEAN'].item() 
ratio = win_mean / bova_mean

ratio


1042.578873935808

In [194]:
repac = pd.read_excel(r'C:\Users\joao.ramalho\Documents\GitHub\BTC\Aluguel\repac.xlsx')

In [172]:
repac['vencimento'] = repac['vencimento'].apply(lambda x: datetime.datetime.strptime(str(x),'%Y%m%d').date())
internas_repac = repac[['str_numcontrato','tipo']].drop_duplicates().dropna()

contratos_internos = list()
for x in internas_repac['str_numcontrato'].unique():
    if internas_repac['str_numcontrato'].tolist().count(x)>1:
        contratos_internos.append(x)

repac = repac[~repac['str_numcontrato'].isin(contratos_internos)]

repac = repac[repac['vencimento']>dt]

repac = repac[['dte_data','str_fundo', 'str_mesa', 'str_estrategia', 'str_numcontrato','vencimento',
       'dbl_quantidade', 'codigo', 'tipo', 'corretora','preco','preco d-1', 'taxa d-1', 'taxa média', 'volume repac', 'volume atual']]

tomador_repac = repac[repac['tipo']=='T']
doador_repac = repac[repac['tipo']=='D']

tomador_repac = tomador_repac[tomador_repac['volume repac']<tomador_repac['volume atual']]
tomador_repac['dif'] = (tomador_repac['volume repac']/tomador_repac['volume atual'])*100
tomador_repac_new = tomador_repac[tomador_repac['dif']<70]
contratos_devol = tomador_repac_new['str_numcontrato'].unique()

tomador_repac_new = tomador_repac_new.groupby(['str_fundo','codigo']).agg({'dbl_quantidade':sum}).reset_index()


doador_repac['dif'] = (doador_repac['volume repac']/doador_repac['volume atual'])*100
doador_repac_new = doador_repac[doador_repac['dif']>200]




Index(['Unnamed: 0', 'dte_data', 'str_fundo', 'str_mercado', 'str_codigo',
       'str_serie', 'str_mesa', 'str_estrategia', 'str_numcontrato',
       'dbl_quantidade', 'codigo', 'tipo', 'preco', 'saldo', 'taxa',
       'modalidade', 'vencimento', 'volume', 'corretora', 'taxa d-1',
       'preco d-1', 'taxa média', 'volume repac', 'volume atual'],
      dtype='object')

,str_numcontrato,tipo
0,2023102500507547200001-2,T
2,2023103100523638370001-1,T
5,2023103100523639310001-1,T
6,2023103100523639300001-1,T
7,2023103100523639290001-1,T
...,...,...
9106,2023102000519719390001-1,T
9108,2023101800507993940001-2,T
9110,2023102500507361990001-2,T
9111,2023110100470898980001-2,D


In [155]:
doador

In [193]:
doador_new

,dte_data,str_fundo,str_mesa,str_estrategia,str_numcontrato,vencimento,dbl_quantidade,codigo,tipo,corretora,preco,preco d-1,taxa d-1,taxa média,volume repac,volume atual,dif
266,2023-11-06,KAPITALO OMEGA PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,2023100400512968050001-1,2023-11-16,-186,CMIG4,D,Bradesco,12.13,12.68,0.04,0.01,-9.433920e+01,-22.5618,418.136851
267,2023-11-06,KAPITALO OMEGA PREV MASTER FIM,Kapitalo 1.0,Bolsa 2,2023100400512968050001-1,2023-11-16,-22,CMIG4,D,Bradesco,12.13,12.68,0.04,0.01,-1.115840e+01,-2.6686,418.136851
268,2023-11-06,KAPITALO ZETA MASTER FIM,Kapitalo 11.1,Bolsa 2,2023101900519088890001-1,2023-12-15,-33242,CMIG4,D,XP,11.73,12.68,0.04,0.01,-1.686034e+04,-3899.2866,432.395567
269,2023-11-06,KAPITALO ZETA MASTER FIM,Kapitalo 1.0,Bolsa 2,2023101900519088890001-1,2023-12-15,-66482,CMIG4,D,XP,11.73,12.68,0.04,0.01,-3.371967e+04,-7798.3386,432.395567
785,2023-11-06,KAPITALO ZETA MASTER FIA,Kapitalo 11.1,CashCarry,2023100300512513200001-1,2023-11-14,-765,RDOR3,D,Bradesco,24.88,23.50,0.43,0.15,-7.730325e+03,-2854.9800,270.766345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7501,2023-11-06,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,2023103100513173850001-2,2023-12-04,-5515,CMIN3,D,Bradesco,5.06,5.69,36.92,0.17,-1.158563e+06,-4744.0030,24421.622878
7502,2023-11-06,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,2023100400513260760001-1,2023-11-16,-8764,CMIN3,D,XP,4.73,5.69,36.92,0.17,-1.841096e+06,-7047.1324,26125.457033
7503,2023-11-06,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,2023100200511882830001-1,2023-11-13,-2766,CMIN3,D,XP,4.82,5.69,36.92,0.12,-5.810669e+05,-1599.8544,36319.986169
7504,2023-11-06,KAPITALO K10 PREV MASTER FIM,Kapitalo 11.1,Bolsa 2,2023092800510781170001-1,2023-11-09,-2852,CMIN3,D,Terra,4.60,5.69,36.92,0.12,-5.991333e+05,-1574.3040,38057.028986


,dte_data,str_fundo,str_mesa,str_estrategia,str_numcontrato,vencimento,dbl_quantidade,codigo,tipo,corretora,preco,preco d-1,taxa d-1,taxa média,volume repac,volume atual,dif
1803,2023-11-06,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,Arbitragem Aluguel,2023110100524182010001-1,2023-12-13,-499127,BRFS3,D,Plural,10.47,12.80,29.83,18.00,-1.905787e+08,-9.406547e+07,202.602144
1791,2023-11-06,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,Box_3pontas,2023110300525014370001-1,2023-12-14,-85000,BRFS3,D,Guide,10.99,12.80,29.83,5.00,-3.245504e+07,-4.670750e+06,694.857143
1789,2023-11-06,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,Box_3pontas,2023110300525014390001-1,2023-12-14,-70000,BRFS3,D,Guide,10.99,12.80,29.83,5.00,-2.672768e+07,-3.846500e+06,694.857143
269,2023-11-06,KAPITALO ZETA MASTER FIM,Kapitalo 1.0,Bolsa 2,2023101900519088890001-1,2023-12-15,-66482,CMIG4,D,XP,11.73,12.68,0.04,0.01,-3.371967e+04,-7.798339e+03,432.395567
1790,2023-11-06,KAPITALO KAPPA MASTER FIM,Kapitalo 11.1,Box_3pontas,2023110300525014380001-1,2023-12-14,-45000,BRFS3,D,Guide,10.99,12.80,29.83,5.00,-1.718208e+07,-2.472750e+06,694.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,2023-11-06,KAPITALO OMEGA PREV MASTER FIM,Kapitalo 1.0,Bolsa 2,2023100400512968050001-1,2023-11-16,-22,CMIG4,D,Bradesco,12.13,12.68,0.04,0.01,-1.115840e+01,-2.668600e+00,418.136851
3362,2023-11-06,KAPITALO ZETA MASTER FIA,Kapitalo 1.0,Bolsa 2,2023100400512966970001-1,2023-11-16,-10,SANB11,D,Bradesco,25.75,27.82,7.42,1.15,-2.064244e+03,-2.961250e+02,697.085352
3361,2023-11-06,KAPITALO ZETA MASTER FIA,Kapitalo 11.1,Bolsa 2,2023100400512966970001-1,2023-11-16,-5,SANB11,D,Bradesco,25.75,27.82,7.42,1.15,-1.032122e+03,-1.480625e+02,697.085352
6398,2023-11-06,KAPITALO ZETA MASTER FIA,Kapitalo 11.1,CashCarry,2023100600514062470001-1,2023-11-20,-2,HYPE3,D,XP,35.54,31.55,0.55,0.12,-3.470500e+01,-8.529600e+00,406.877228


In [223]:

holidays_br = workdays.load_holidays('BR')
holidays_b3 = workdays.load_holidays('B3')
dt = datetime.date.today()
dt_1 = workdays.workday(dt, -1, holidays_b3)



dt_2 = workdays.workday(dt, -2, holidays_b3)

db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user='k_geral', password='Kapitalo@01')    
query=f"select dte_data,dbl_lote,dbl_pu from tbl_auxboletas1 where  dte_data>='2023-07-01' and str_corretora='Liquidez' and str_estrategia='Arbitragem Aluguel' and str_mercado='Acao' " 
query_off=f"select dte_data,dbl_qtetrade,dbl_putrade from tbl_boletasexternas where  dte_data>='2023-07-01' and str_broker like '%BGC%' and str_estrategia='Arbitragem Aluguel' and str_mercado='Acao' " 
df_on =pd.read_sql(query,db_conn_test)
df_off =pd.read_sql(query_off,db_conn_test)

db_conn_test.close()



In [239]:
df_off.sort_values('dte_data')

,dte_data,dbl_qtetrade,dbl_putrade,Abs
13,2023-07-03,100.0,9.290000,100.0
22,2023-07-03,86635.0,7.943147,86635.0
17,2023-07-03,120.0,8.245000,120.0
16,2023-07-03,-5400.0,9.373685,5400.0
15,2023-07-03,-17515.0,13.652552,17515.0
...,...,...,...,...
174,2023-09-13,135866.0,7.498537,135866.0
173,2023-09-13,-8884.0,8.443613,8884.0
172,2023-09-13,1086.0,8.390737,1086.0
167,2023-09-13,-78708.0,7.582340,78708.0


In [226]:
df_on['Abs'] = df_on['dbl_lote'].apply(lambda x: abs(x))
df_off['Abs'] = df_off['dbl_qtetrade'].apply(lambda x: abs(x))

In [231]:
volume_on = (df_on['Abs']*df_on['dbl_pu']).sum()

volume_off = (df_off['Abs']*df_off['dbl_putrade']).sum()

In [238]:
volume_off

99766488.98496819

In [233]:
correta_off = df_off['Abs'].sum()*0.0032
correta_on = volume_on*(0.5/100)*(1-98.5/100)

In [235]:
print(correta_on)
print(correta_off)

36582.67070174812
32136.624


In [144]:
df['tipo'] = df['str_serie'].apply(lambda x: x.split('-')[1])
df['test'] = df.apply(lambda row: False if ((row['tipo']=='T')& (row['dbl_quantidade']<0) ) else False if ((row['tipo']=='D')& (row['dbl_quantidade']>0) ) else True,axis=1 )

In [145]:
ajuste = df[df['test']==False]

In [146]:
ajuste

,dte_data,str_fundo,str_mercado,str_codigo,str_serie,str_mesa,str_estrategia,str_numcontrato,dbl_quantidade,tipo,test
4787,2023-07-18,KAPITALO KAPPA MASTER FIM,Emprestimo RV,MRVE,"MRVE3-T-13,03-20230823-BALCAO",Kapitalo 11.1,CashCarry,2023071400482046370001-1,-164.0,T,False
4794,2023-07-18,KAPITALO KAPPA MASTER FIM,Emprestimo RV,MRVE,"MRVE3-T-11,11-20230821-BALCAO",Kapitalo 11.1,CashCarry,2023071200480878260001-1,-667.0,T,False
7303,2023-07-18,KAPITALO KAPPA MASTER FIM,Emprestimo RV,BRFS,BRFS3-D-20-20230818-BALCAO,Kapitalo 11.1,MM,2023071100480668670001-1,814.0,D,False


In [147]:
contratos = ajuste['str_numcontrato'].unique()

In [151]:
ajuste.to_clipboard()

In [149]:
target = df.loc[(df['test']==True)&(df['str_numcontrato'].isin(contratos))]

In [150]:
target

,dte_data,str_fundo,str_mercado,str_codigo,str_serie,str_mesa,str_estrategia,str_numcontrato,dbl_quantidade,tipo,test
4265,2023-07-18,KAPITALO KAPPA MASTER FIM,Emprestimo RV,MRVE,"MRVE3-T-13,03-20230823-BALCAO",Kapitalo 11.1,MM,2023071400482046370001-1,164.0,T,True
4789,2023-07-18,KAPITALO KAPPA MASTER FIM,Emprestimo RV,MRVE,"MRVE3-T-13,03-20230823-BALCAO",Kapitalo 11.1,Box_3pontas,2023071400482046370001-1,164.0,T,True
4793,2023-07-18,KAPITALO KAPPA MASTER FIM,Emprestimo RV,MRVE,"MRVE3-T-11,11-20230821-BALCAO",Kapitalo 11.1,MM,2023071200480878260001-1,763.0,T,True
4795,2023-07-18,KAPITALO KAPPA MASTER FIM,Emprestimo RV,MRVE,"MRVE3-T-11,11-20230821-BALCAO",Kapitalo 11.1,Box_3pontas,2023071200480878260001-1,931.0,T,True
7304,2023-07-18,KAPITALO KAPPA MASTER FIM,Emprestimo RV,BRFS,BRFS3-D-20-20230818-BALCAO,Kapitalo 11.1,CashCarry,2023071100480668670001-1,-7489.0,D,True
7305,2023-07-18,KAPITALO KAPPA MASTER FIM,Emprestimo RV,BRFS,BRFS3-D-20-20230818-BALCAO,Kapitalo 11.1,Box_3pontas,2023071100480668670001-1,-30692.0,D,True
7306,2023-07-18,KAPITALO KAPPA MASTER FIM,Emprestimo RV,BRFS,BRFS3-D-20-20230818-BALCAO,Kapitalo 11.1,Bolsa 2,2023071100480668670001-1,-4.0,D,True
7307,2023-07-18,KAPITALO KAPPA MASTER FIM,Emprestimo RV,BRFS,BRFS3-D-20-20230818-BALCAO,Kapitalo 11.1,Arbitragem Aluguel,2023071100480668670001-1,-62629.0,D,True


In [53]:
target['dvd'] = target['str_numcontrato'].apply(lambda x: target['str_numcontrato'].tolist().count(x)  )

In [54]:
target['mesa target'] = target['str_mesa']
target['estrategia target'] = target['str_estrategia']

In [55]:
ajuste.merge(target,on=['str_fundo','str_numcontrato'],how='left').to_clipboard()

In [56]:
ajuste

,dte_data,str_fundo,str_mercado,str_codigo,str_serie,str_mesa,str_estrategia,str_numcontrato,dbl_quantidade,tipo,test
2001,2023-07-13,KAPITALO KAPPA MASTER FIM,Emprestimo RV,SLCE,"SLCE3-T-0,06-20230724-BALCAO",Kapitalo 11.1,CashCarry,2023061400472446860001-1,-88.000,T,False
2050,2023-07-13,KAPITALO KAPPA MASTER FIM,Emprestimo RV,SANB,"SANB11-T-6,61-20230727-BALCAO",Kapitalo 11.1,Box_3pontas,2023061900473579850001-1,-811.517,T,False
2521,2023-07-13,KAPITALO KAPPA MASTER FIM,Emprestimo RV,VIIA,"VIIA3-T-3,64-20230817-BALCAO",Kapitalo 11.1,MM,2023071000480178220001-1,-164.000,T,False
3860,2023-07-13,KAPITALO KAPPA MASTER FIM,Emprestimo RV,CSAN,"CSAN3-T-0,02-20230726-BALCAO",Kapitalo 11.1,MM,2023061600473045150001-1,-4977.018,T,False
3862,2023-07-13,KAPITALO KAPPA MASTER FIM,Emprestimo RV,CSAN,"CSAN3-T-0,02-20230726-BALCAO",Kapitalo 11.1,CashCarry,2023061600473045150001-1,-1.660,T,False
3863,2023-07-13,KAPITALO KAPPA MASTER FIM,Emprestimo RV,CSAN,"CSAN3-T-0,02-20230726-BALCAO",Kapitalo 11.1,Bolsa 2,2023061600473045150001-1,-1.661,T,False
3869,2023-07-13,KAPITALO KAPPA MASTER FIM,Emprestimo RV,CSAN,"CSAN3-T-0,01-20230724-BALCAO",Kapitalo 11.1,Bolsa 2,2023061400472443500001-1,-178.000,T,False
